In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras_tuner as kt
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras import Sequential, layers
from keras.layers import Dense, Dropout
from pickle import load

In [ ]:
# raw_data = pd.read_csv(r"../data/processed_data/undersampled_train.csv", index_col=0)
raw_data = pd.read_csv(r"../data/processed_data/complete_data.csv", index_col=0)

In [ ]:
x = raw_data.drop(["TARGET"], axis=1)
y = raw_data.filter(["TARGET"], axis=1)

scaler = scaler = load(open(r"scaler.pkl", "rb"))  # MinMaxScaler often made values = 0.0, which = lost features in training
x_scaled = scaler.transform(x)
y = y.values  # we dont need to scale our binary classification

# x_scaled = x_scaled.reshape(x_scaled.shape[0], x_scaled.shape[1], )  # data needs to be 3D for a CNN

x_scaled.shape, y.shape

In [ ]:
# split the dataset into 50/50 - 50 for tuning, 50 for kfold validation
split = 0.5

hp_x, hp_y = x_scaled[:int(len(x_scaled) * split)], y[:int(len(y) * split)] # hyperparameter set
kf_x, kf_y = x_scaled[int(len(x_scaled) * split):], y[int(len(y) * split):] # kfold set


In [ ]:
def model_builder(hp):
    model = Sequential()

    # input layer
    model.add(Dense(units=x_scaled.shape[1], input_shape=(x_scaled.shape[1],)))

    # let the model decide how many layers it wants to have
    for i in range(hp.Int("num_dense_layers", min_value=2, max_value=5, step=1)):
        model.add(
            Dropout(hp.Float("dropout_" + str(i), min_value=0, max_value=0.5, step=0.1))
        )
        model.add(
            Dense(
                units=hp.Int("layer_" + str(i), min_value=32, max_value=1024, step=64),
                activation=hp.Choice("act_" + str(i), ["relu", "sigmoid"]),
            )
        )

    # output shape of the model the same as the number of features
    model.add(
        Dense((1), activation="sigmoid"),
    )

    # compile the model
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.AUC(), "accuracy"],
    )

    return model


In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=2)

In [ ]:
project = "BayOpt_v1.04"

tuner = kt.BayesianOptimization(
    model_builder,
    objective=kt.Objective("accuracy", direction="max"),
    max_trials=20, # I could stop it earlier, or even let it run forever but, my computer isn't powerful enough
    directory=f"../logs/{project}",
    project_name=project,
)


In [ ]:
tuner.search(
    hp_x,
    hp_y,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    callbacks=[earlyStop],
)


In [ ]:
hpModel = tuner.get_best_models()[0]
hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

hpModel.summary(), hyperparameters.values

In [ ]:
trained_model = hpModel.fit(
    x_scaled, y, epochs=10, batch_size=32, callbacks=earlyStop
)

In [ ]:
hpModel.save(f"neural_network/{project}")

In [ ]:
model_performance = hpModel.evaluate(x_scaled, y)
print(f"Loss: {model_performance[0]}\nAccuracy: {model_performance[1]}")

In [ ]:
fig, (ax1) = plt.subplots(figsize=(6, 4))
fig.suptitle("Evaluation of Model Training")
ax2 = ax1.twinx()

ax1.plot(trained_model.history["accuracy"], label="Accuracy")
ax1.set_ylabel("Accuracy")
ax1.set_xlabel("Epoch")

ax2.plot(trained_model.history["loss"], color="r", label="Loss")
ax2.set_ylabel("Loss")
ax2.set_xlabel("Epoch")

fig.legend()

# plt.tight_layout()
# plt.savefig(r"../images/neural_network-training_evaluation")